# **Advanced Feature Engineering in BQML**

**Learning Objectives**

1. Evaluate the model
2. Extract temporal features, feature cross temporal features
3. Apply `ML.FEATURE_CROSS` to categorical features
4. Create a Euclidian feature column, feature cross coordinate features
5. Apply the `BUCKETIZE` function, `TRANSFORM` clause, L2 Regularisation

## **Introduction**

In this lab, we utilise feature engineering to improve the prediction of the fare amount for a taxi ride in New York City. We will use BigQuery ML to build a taxifare prediction model, using feature engineering to improve and create a final model.

In this notebook, we perform a feature cross using BigQuery's `ML.FEATURE_CROSS`, derive coordinate features, feature cross coordinate features, clean up the code, apply the `BUCKETIZE` function, the `TRANSFORM` clause, L2 Regularisation, and evaluate model performance throughout the process.

## **Set up environment variables and load necessary libraries**

In [ ]:
# Install Google Cloud BigQuery
!pip install --user google-cloud-bigquery=1.25.0

In [ ]:
%%bash

export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is "$PROJECT

## **The source data set**

Our data set is hosted in BigQuery. The taxi fare data is a publically available data set, meaning anyone with a GCP account has access.

The Taxi Fare data set is relatively large at 55 million training rows, but simple to understand, with only 6 features. The `fare_amount` is the target, the continuous value we'll train a model to predict.

## **Create a BigQuery data set**

A BigQuery data set is a container for tables, views, and models built with BigQuery ML. Let's create one called **feat_eng**. Weel do the same for a GCS bucket for our project too.

In [ ]:
%%bash

# Create a BigQuery data set for feat_eng if it doesn't exist
datasetexists=$(bq ls -d | grep -w feat_eng)

if [ -n "$datasetexists"]; then
    echo -e "BigQuery dataset already exists, let's not recreate it"

else
    echo "Creating BigQuery dataset titled: feat_eng"
    
    bq --location=US mk --dataset \
        --description "Taxi Fare" \
        $PROJECT:feat_eng
    echo "\nHere are your current datasets:"
    bq ls
fi

## **Creating the training table**

Since there is already a publicly available data set, we can simply create the training data table using this raw input data. Note the `WHERE` clause in the below query: This clause allows us to TRAIN a portion of the data (e.g. one hundred thousand rows versus one million rows), which keeps our query costs down.

In [ ]:
%%bigquery

# Creating the table in our data set
CREATE OR REPLACE TABLE
    feat_eng.feateng_training_data AS
SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    passenger_count * 1.0 AS passengers,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat
FROM
    `nyc-tlc.yellow.trips`
WHERE
    MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 10000) = 1
    AND fare_amount >= 2.5
    AND passenger_count > 0
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45

## **Verify table creation**

Verify that you created the data set

In [ ]:
%%bigquery

# LIMIT 0 is a free query; this allows us to check that the table exists
SELECT
    *
FROM
    feat_eng.feateng_training_data
LIMIT
    0

## **Baseline Model**

### **Create the baseline model**

Next, you create a linear regression basline model with no feature engineering. Recall that a model in BigQuery ML represents what an ML system has learned from the training data. A baseline model is a solution to a problem without applying any ML techniques.

When creating a BQML model, you must specify the model type (in our case linear regression) and the input label (`fare_amount`). Note also that we are using the training data table as the data source.

Now we create the SQL statement to create the baseline model.

In [ ]:
%%biquery

# Creating the baseline model
CREATE OR REPLACE MODEL
    feat_eng.baseline_model OPTIONS(model_type="linear_reg",
                                    input_label_cols=["fare_amount"])
    AS
SELECT
    fare_amount,
    passengers,
    pickup_datetime,
    pickuplon,
    pikcuplat,
    dropofflon,
    dropofflat
FROM
    feat_eng.feateng_training_data

### **Evaluate the baseline model**

Note that BigQuery automatically split the data we gave it, and trained on only a part of the data set and used the rest for evaluation. After creating your model, you evaluate the performance of the regressor using the `ML.EVALUATE` function. The `ML.EVALUATE` function evaluates the predicted values against the actual data.

Review the learning and eval statistics for the `baseline_model`.

In [ ]:
%%bigquery

# Eval statistics on the held out data.
SELECT
    *,
    SQRT(loss) AS rmse
FROM
    ML.TRAINING_INFO(MODEL feat_eng.baseline_model)

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    *
FROM
    ML.EVALUATE(MODEL feat_eng.baseline_model)

**NOTE:** Because you performed a linear regression, the results include the following columns:

- `mean_absolute_error`
- `mean_squared_error`
- `mean_squared_log_error`
- `median_absolute_error`
- `r2_score`
- `explained_variance`

**Mean squared error (MSE)** $-$ Measures the difference between the values our model predicted using the test set and the actual values. You can also think of it as the distance between your regression (best fit) line and the predicted values.

**Root mean squared error (RMSE)** $-$ The primary evaluation metric for this ML problem is the root mean squared error. RMSE measures the difference between the predictions of a model, and the observed values. A large RMSE is equivalent to a large average error, so smaller values of RMSE are better. One nice property of RMSE is that the error is given in the units being measured, so you can tell very directly how incorrect the model might be on unseen data.

**R2**: An important metric in the evaluation results in the R2 score. The R2 score is a statistical measure that determines if the linear regression predictions approxumate the actual data. $0$ indicates that the model explains none of the variability of the response data around the mean. $1$ indicates that the model explains all the variability of the response data around the mean.

Next, we write a SQL query to take the `SQRT()` of the mean squared error as your loss metric for evaluation for the `benchmark_model`.

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL feat_eng.baseline_model)

## **Model 1 $-$ EXTRACT `dayofweek` from the `pickup_datetime` feature**

- As you recall, `dayofweek` is an `enum` representing the 7 days of the week. This factory allows the `enum` to be obtained from the `int` value. The `int` value follows the ISO-8601 standard, from `1` (Monday) to `7` (Sunday).
- If you were to extract the `dayofweek` from `pickup_datetime` using BigQuery SQL, the datatype returned would be integer

Next, we create a model titled `model_1` from the baseline model and extract out the `DayOfWeek`.

In [ ]:
%%bigquery

# Creating the model from baseline model and extract the Days of Week
CREATE OR REPLACE MODEL
    feat_eng.model_1 OPTIONS (model_type="linear_reg",
                              input_label_cols=["fare_amount"]) AS
SELECT
    fare_amount,
    passengers,
    pickup_datetime,
    EXTRACT(DAYOFWEEK FROM pickup_datetime) AS dayofweek,
    pickuplon,
    pickuplat,
    dropofflon,
    dropofflat
FROM
    feat_eng.feateng_training_data

Next, two distinct SQL statements show the TRAINING and EVALUATION metrics of `model_1`.

In [ ]:
%%bigquery

# Use `ML.TRAINING_INFO` function which allows you to see information about the training iterations of a model
SELECT
    *,
    SQRT(loss) AS rmse
FROM
    ML.TRAINING_INFO(MODEL feat_eng.model_1)

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics.
SELECT
    *
FROM
    ML.EVALUATE(MODEL feat_eng.model_1)

In [ ]:
%% bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL feat_eng.model_1)

## **Model 2 $-$ EXTRACT `hourofday` from the `pickup_datetime` feature**

As you recall, `pickup_datetime` is stored as a TIMESTAMP, where the Timestamp format is retrieved in the standard output format $-$ year-month-day hour:minute:second. `hourofday` returns the integer number representing the hour number of the given date.

`hourofday` is best thought of as a discrete ordinal variable (and not a categorical feature), as the hours can be ranked (e.g. there is a natural ordering of the values). `hourofday` has an added characteristic of being cyclic, since 12am follows 11pm and precedes 1am.

Next, we create a model titled `model_2` and EXTRACT the `hourofday` from the `pickup_datetime` feature to improve our model's rmse.

In [ ]:
%%bigquery

# Creating the model from baseline model and extract the hours of day
CREATE OR REPLACE MODEL
    feat_eng.model_2 OPTIONS (model_type="linear_reg",
                              input_label_cols=["fare_amount"]) AS
SELECT
    fare_amount,
    passengers,
    EXTRACT(DAYOFWEEK FROM pickup_datetime) AS dayofweek,
    EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
    pickuplon,
    pickuplat,
    dropofflon,
    dropofflat
FROM
    `feat_eng.feateng_training_data`

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    *
FROM
    ML.EVALUATE(MODEL feat_eng.model_2)

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL feat_eng.model_2)

## **Model 3 $-$ Feature cross `dayofweek` and `hourofday` using `CONCAT`**

First, let's allow the model to learn traffic patterns by creating a new feature that combines the time of day and day of week. This is called a *feature cross*

Note: BQML by default assumes that numbers are numeric features, and strings are categorical features. We need to convert both the `dayofweek` and `hourofday` features to strings because the model (Neural Network) will automatically treat any integer as a numerical value rather than a categorical value. Thus, if not cast as string, the `dayofweek` feature will be interpreted as numeric values (e.g. $1, 2, 3, 4, 5, 6, 7$) and `hourofday` will also be interpreted as numeric values (e.g. the day begins at midnight, `00:00`, and the last minute of the day begins at `23:59` and ends at `24:00`). As such, there is no way to distinguish the *feature cross* of `hourofday` and `dayofweek` *numerically*. Casting the `dayofweek` and `hourofday` as strings ensures that each element will be treated like a label and will get its own coefficient associated with it.

Create the SQL statement to feature cross the `dayofweek` and `hourofday` using the CONCAT function. Name the model `model_3`.

In [ ]:
%%bigquery

# Using `CONCAT` function to feature cross the `dayofweek` and `hourofday`
CREATE OR REPLACE MODEL
    feat_eng.model_3 OPTIONS(model_type="linear_reg",
                             input_label_cols=["fare_amount"]) AS
SELECT
    fare_amount,
    passengers,
    CONCAT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING), 
           CAST (EXTRACT(HOUR FROM pickup_datetime) AS STRING)) AS hourofday,
    pickuplon,
    pickuplat,
    dropofflon,
    dropofflat
FROM
    `feat_eng.feateng_training_data`

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    *
FROM
    ML.EVALUATE(MODEL feat_eng.model_3)

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL feat_eng.model_3)

## **Model 4 $-$ Apply the `ML.FEATURE_CROSS` clause to categorical features**

BigQuery ML now has `ML.FEATURE_CROSS`, a pre-processing clause that performs a feature cross.

- `ML.FEATURE_CROSS` generates a `STRUCT` feature with all combinations of crossed categorical features, except for 1-degree items (the original features) and self-crossing items.
- Syntax: `ML.FEATURE_CROSS(STRUCT(features), degree)
- `features`: Categorical features to be crossed separated by `,`. The maximum number of input features is 10. An unnamed feature is not allowed in features. Duplicates are not allowed in features
- `degree` (optional): The highest degree of all combinations. Degree should be in the rage of `[1, 4]`. Default to 2.

Output: The function outputs a STRUCT of all combinations except for 1-degree items and self-crossing items, with field names as concatenation of original feature names and values as the concatenation of the column string values.

Here, we examine the components of `ML.FEATURE_CROSS`.

In [ ]:
%%bigquery

# Using the `ML.FEATURE_CROSS` clause
CREATE OR REPLACE MODEL 
    feat_eng.model_4 OPTIONS(model_type="linear_reg",
                             input_label_cols=["fare_amount"]) AS
SELECT
    fare_amount,
    passengers,
    ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
                            CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr,
    pickuplon,
    pickuplat,
    dropofflon,
    dropofflat

FROM `feat_eng.feateng_training_data`

Next, two distinct SQL statements show the TRAINING and EVALUATION metrics of `model_4`

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL feat_eng.model_4)

## **Sliding down the slope towards a loss minimum (reduced taxi fare)!**

- Our fourth model gives us an RMSE of 9.65 for estimating fares. Recall our heuristic baseline was 8.29. This may be the result of feature cross. Let's apply more feature engineering techniques to see if we can't get this loss metric lower!

## **Model 5 $-$ Feature cross coordinate features to create a Euclidean feature**

Pickup coordinate:

- `pickup_longitude` AS `pickuplon`
- `pickup_latitude` AS `pickuplat`

Dropoff coordinate:

- `droppoff_longitude` AS `dropofflon`
- `dropoff_latitude` AS `dropofflat`

### **Coordinate Features**

- The pick-up and drop-off longitude and latitude data are crucial to predicting the fare amount as fare amounts in NYC taxis are largely determined by the distance traveled. As such, we need to teach the model the Euclidean distance bewteen the pick-up and drop-off points.

- Recall that latitude and longitude allow us to specify any location on Earth using a set of coordinates. In our training data set, we restricted our data points to only pick ups and drop offs within NYC. New York City has an appropriate longitude range of $-74.05$ to $-73.75$ and a latitude range of $40.63$ to $40.85$.

- The data set contains information regarding the pick up and drop off coordinates. However, there is no information regarding the distance between the pick up and drop off points. Therefore, we create a new feature that calculates the distance between each pair of pickup and drop off points. We can do this using the Euclidean Distance, which is the straight-line distance between any two coordinate points

- We need to convert those coordinates into a single column of a spatial data type. We will use the `ST_Distance` and the `ST_GeogPoint` functions

- `ST_Distance(geography_1, geography_2)`: Returns the shortest distance in meters between two non-empy `GEOGRAPHY`s (e.g. between two spatial objects)

- `ST_GeogPoint(longitude, latitude)`: Creates a `GEOGRAPHY` with a single point. `ST_GEOPOINT` creates a point from the specified `FLOAT64` `latitude` and `longitude` parameters and returns that point in a `GEOGRAPHY` value

Next we convert the feature coordinates into a single column of a spatial data type. Use the `ST_Distance` and the `ST_GeogPoint` functions

In [ ]:
%%bigquery

# Convert the feature coordinates into a single column of a `spatial` data type
CREATE OR REPLACE MODEL
    feat_eng.model_5 OPTIONS(model_type="linear_reg",
                             input_label_cols=["fare_amount"]) AS
SELECT
    fare_amount,
    passengers,
    ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
                            CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr,
    ST_Distance(ST_GeogPoint(pikcuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) AS euclidean
FROM
    `feat_eng.feateng_training_data`

Next, two distinct SQL statements show metrics for `model_5`

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    *
FROM
    ML.EVALUATE(MODEL feat_eng.model_5)

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL feat_eng.model_5)

## **Model 6 $-$ Feature cross pick-up and drop-off locations features**

In this section, we feature cross the pick-up and drop-off locations so that the model can learn pick-up-drop-off pairs that will require tolls.

This step takes the geographic point corresponding to the pickup point and grids to a $0.1$-degree-latitude/longitude grid (approximately $8$ km $\times$ $11$ km in New York $-$ we should experiment with finer grid resolutions as well). Then, it concatenates the pickup and dropoff grid points to learn *corrections* beyond the Euclidean distance associated with pairs of pickup and dropoff locations.

Because the lat and lon by themselves don't have meaning, but only in conjunction, it may be useful to treat the  fields as a pair instead of just using them as numeric values. However, lat and lon are continuous numbers, so we have to discretise them first. That's what `SnapToGrid` does.

- `ST_SnapToGrid(geography_expression, grid_size)`: Returns the input `GEOGRAPHY`, where each vertex has been snapped to a longitude/latitude grid. The grid size is determined by the `grid_size` parameter which is given in degrees

**REMINDER**: The `ST_GeogPoint` creates a `GEOGRAPHY` with a single point. `ST_GeogPoint` creates a point from the specified `FLOAT64` longitude and latitude parameters and returns that point in a `GEOGRAPHY` value. The `ST_Distance` function returns the minimum distance beteween two spatial objects. It also returns meters for geographies and SRID units for geometrics.

In [ ]:
%%bigquery

# Use feature cross for the pick-up and drop-off locations features
CREATE OR REPLACE MODEL
    feat_eng.model_6 OPTIONS(model_type="linear_reg",
                             input_label_cols=["fare_amount"]) AS
SELECT
    fare_amount,
    passengers,
    ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
                            CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr,
    ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) AS euclidean,
    CONCAT(ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickuplon, pickuplat), 0.01)),
           ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropofflon, dropofflat), 0.01))) AS pickup_and_dropoff
FROM
    `feat_eng.feateng_training_data`

Next, we evaluate `model_6`

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    *
FROM
    ML.EVALUATE(MODEL feat_eng.model_6)

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL feat_eng.model_6)

## **BQML's Pre-processing functions**

Here are some of the preprocessing functions in BigQuery ML

- `ML.FEATURE_CROSS(STRUCT(features))` does a feature cross of all the combinations
- `ML.POLYNOMIAL_EXPAND(STRUCT(features), degree)` creates $x$, $x^2$, $x^3$, etc.
- `ML.BUCKETIZE(feature, split_points)` where `split_points` is an array

## **Model 7 $-$ Apply the `BUCKETIZE` function**

`BUCKETIZE` is a pre-processing function that creates *buckets* (e.g. bins) $-$ e.g. it bucketises a continuous numerical feature into a string feature with bucket names as the value

- `ML.BUCKETIZE(feautre, split_points)`
- `feature`: A numerical column
- `split_points`: Array of numerical points to split the continuous values in feature into buckets. With $n$ split points (`s1`, `s2`,..., `sn`), there will be $n+1$ buckets generated.
- output: The function outputs a `STRING` for each row, which is the bucket name
- Currently, our model uses the `ST_GeogPoint` function to derive the pickup and dropoff feature. In this lab, we use the `BUCKETIZE` function to create the pickup and dropoff feature

Next, apply the `BUCKETIZE` function to `model_7`

In [ ]:
%%bigquery

# Use `BUCKETIZE` function
CREATE OR REPLACE MODEL
    feat_eng.model_7 OPTIONS(model_type="linear_reg",
                             input_label_cols=["fare_amount"]) AS
SELECT
    fare_amount,
    passengers,
    ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) AS euclidean,
    ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
                            CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr,
    CONCAT(ML.BUCKETIZE(pickuplon, GENERATE_ARRAY(-78, -70, 0.01)),
           ML.BUCKETIZE(pickuplat, GENERATE_ARRAY(37, 45, 0.01)),
           ML.BUCKETIZE(dropofflon, GENERATE_ARRAY(-78, -70, 0.01)),
           ML.BUCKETIZE(dropofflat, GENERATE_ARRAY(37, 45, 0.01))) AS pickup_and_dropoff
FROM
    `feat_eng.feateng_training_data`

Next, we evaluate `model_7`

In [ ]:
%%bigquery

# Use `ML.TRAINING_INFO` function which allows you to see information about the training iterations of a model
SELECT
    *,
    SQRT(loss) AS rmse
FROM
    ML.TRAINING_INFO(MODEL feat_eng.model_7)

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    *
FROM
    ML.EVALUATE(MODEL feat_eng.model_7)

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL feat_eng.model_7)

## **Final Model: Apply the `TRANSFORM` clause and L2 Regularisation**

Before we perform our prediction, we should encapsulate the entire feature set in a `TRANSFORM` clause. BigQuery ML now supports defining data transformations during model creation, which will be automatically applied during prediction and evaluation. This is done through the `TRANSFORM` clause in the existing `CREATE MODEL` statement. By using the `TRANSFORM` clause, user specified transforms during training will be automatically applied during model serving (prediction, evaluation, etc.)

In our case, we are using the `TRANSFORM` clause to separate out the raw input data from the *transformed* features. The input columns of the `TRANSFORM` clause are the `query_expr` (`AS SELECT` part). The output columns of the `TRANSFORM` from `select_list` are used in training. These transformed columns are post-processed with standardisation for numerics and one-hot encoding for categorical variables by default.

The advantage of encapsulating features in the `TRANSFORM` clause is the client code doing the `PREDICT` doesn't chage, e.g. our model improvement is transparent to client code. Note that the `TRANSFORM` clause `MUST` be placed after the `CREATE` statement.

### **L2 Regularisation**

Sometimes, the training RMSE is quite reasonable, but the evaluation RMSE illustrates more error. Given the severity of the delta between the `EVALUATION RMSE` and the `TRAINING RMSE`, it may be an indication of *overfitting*. When we do feature crosses, we run into the risk of overfitting (for example, when a particular day-hour combo doesn't have enough taxi rides).

Overfitting is a phenomenon that occurs when an ML model is tailored to a particular data set and is unable to generalise to other data sets. This usually happens in complex models, like deep neural networks. Regularisation is a process of introducing additional information in order to prevent overfitting.

Therefore, we will apply L2 regularisation to the final model. As a reminder, a regression model that uses the **L1 regularisation** technique is called **Lasso regression** while a regression model that uses the **L2 regularisation** technique is called **Ridge regression**. The key difference between these two is the penalty term. Lasso shrinks the less important feature's coefficient to $0$, thus removing some features altogether. Ridge regression adds *squared magnitude* of coefficient as a penalty term to the loss function.

In other words, **L1 limits the size of the coefficients**. L1 can yield sparse models (i.e. models with few coefficients). Some coefficients can become $0$ and eliminated.

**L2 will not yield sparse models** and all coefficients are shrunk by the same factor (none are eliminated).

The regularisation terms are *constraints* by which an optimisation algorithm must *adhere to* when minimising the loss function, apart from having to minimise the error between the true $y$ and the predicted $\hat{y}$. This in turn reduces model complexity, making our model simpler. A simpler model can reduce the chances of overfitting.

Apply the `TRANSFORM` clause and L2 regularisation to the final model.

In [ ]:
%%bigquery

# Use the `TRANSFORM` clause
CREATE OR REPLACE MODEL
    feat_eng.final_model
    TRANSFORM(fare_amount,
              ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) AS euclidean,
              ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
                                      CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr,
              CONCAT(ML.BUCKETIZE(pickuplon, GENERATE_ARRAY(-78, -70, 0.01)),
                     ML.BUCKETIZE(pickuplat, GENERATE_ARRAY(37, 45, 0.01)),
                     ML.BUCKETIZE(dropofflon, GENERATE_ARRAY(-78, -70, 0.01)),
                     ML.BUCKETIZE(dropofflat, GENERATE_ARRAY(37, 45, 0.01))) AS pickup_and_dropoff) 
    OPTIONS(input_label_cols=["fare_amount"],
            model_type="linear_reg",
            l2_reg=0.1) AS
SELECT
    *
FROM
    feat_eng.feateng_training_data

Next, we evaluate the final model.

In [ ]:
%%bigquery

# Use `ML.TRAINING_INFO` function which allows you to see information about the training iterations of a model
SELECT
    *,
    SQRT(loss) AS rmse
FROM
    ML.TRAINING_INFO(MODEL feat_eng.final_model)

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    *
FROM
    ML.EVALUATE(MODEL feat_eng.final_model)

In [ ]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL feat_eng.final_model)

## **Predictive Model**

Now that you have evaluated your model, the next step is to use it to predict an outcome. You use your model to predict the taxifare amount. The `ML.PREDICT` function is used to predict results using your model `feat_eng.final_model`

Since this is a regression model (predicting a continuous numerical value), the best way to see how it performed is to evaluate the difference between the value predicted by the model and the baseline score. We can do this with an `ML.PREDICT` query.

Apply the `ML.PREDICT` function

In [ ]:
%%bigquery

# Use `ML.PREDICT` function to predict outcomes using the model
SELECT
    *
FROM
    ML.PREDICT(MODEL feat_eng.final_model,
              (SELECT
               -73.982683 AS pickuplon,
               40.742104 AS pickuplat,
               -73.983766 AS dropofflon,
               40.755174 AS dropofflat,
               3.0 AS passengers,
               TIMESTAMP("2019-06-03 04:21:29.769443 UTC") AS pickup_datetime))